# Schlumberger Sounding and Middle Gradient Survey
------------
### Lillian Hanson, Schlumberger (Group A)
* Due Date: October 22, 2025

In [22]:
from dc_utils.dc_sounding import DCSoundingInteract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data processing
1) Each group should share their data sets with the entire class. I would suggest sharing data in a format that lists the locations of A, B, M, and N electrodes, and the measured normalized voltage. Possibly:
|A_east | A_north | B_east| B_north| M_east| M_north| N_east| N_north| V/I | STD|
|-------|---------|-------|--------|-------|--------|-------|--------|-----|----|
| #| #| #| #| #| #| #| #| #| # |
| #|# | #| #| #| #| #| #| #| # |



In [12]:
# import data sets
schlum = pd.read_csv('data/08-data/Lab08-SchlumDogz-Combined-Data(in).csv')
schlum = pd.DataFrame(schlum)
schlum = schlum.drop(16)

schlum

,A_east,A_north,B_east,B_north,M_east,M_north,N_east,N_north,V/I (ohms),I,SD (%)
0,481026.6498,4400176.074,481028.2797,4400177.280,NaN,NaN,NaN,NaN,2.436000,200.0,0.0
1,481026.3057,4400175.849,481028.6039,4400177.452,NaN,NaN,NaN,NaN,1.192000,200.0,0.0
2,481025.8801,4400175.570,481029.0981,4400177.748,NaN,NaN,NaN,NaN,0.583500,200.0,0.0
3,481025.0312,4400174.903,481029.9168,4400178.322,NaN,NaN,NaN,NaN,0.230600,200.0,0.0
4,481025.0312,4400174.903,481029.9168,4400178.322,NaN,NaN,NaN,NaN,0.678100,200.0,0.0
5,481024.2202,4400174.301,481030.7442,4400178.930,NaN,NaN,NaN,NaN,0.115700,200.0,0.0
6,481024.2202,4400174.301,481030.7442,4400178.930,NaN,NaN,NaN,NaN,0.334600,200.0,0.1
7,481022.5631,4400173.180,481032.3755,4400180.063,NaN,NaN,NaN,NaN,0.108500,200.0,0.1
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049960,200.0,0.2
9,481019.3355,4400170.898,481035.6675,4400182.364,NaN,NaN,NaN,NaN,0.027230,200.0,0.0


2. Calculate the apparent resistivity for each data point.

$$ \rho_a = 2\pi\frac{V}{I}\left(\frac{1}{R_{AM}} -\frac{1}{R_{BM}} - \frac{1}{R_{AN}} + \frac{1}{R_{BN}}\right)^{-1} $$


In [3]:
## Calculate apparent resistivity

schlum_locs = pd.read_csv('data/08-data/lab08-schlum-locations(Sheet1).csv')
A_x = schlum_locs['A']
B_x = schlum_locs['B']
M_x = schlum_locs['M']
N_x = schlum_locs['N']

3. Plot the Schlumberger sounding data set with the following two axes, both on a log scale:
    * AB/2 on the vertical axis, increasing downwards.
    * Apparent resistivity on the horizontal axis.

4. For the middle gradient survey make two 2D maps of your data. One for the normalized voltage (V/I), and another for the apparent resistivity. Use the center point of the M-N electrode dipole pair as the location for these points.

In [27]:
gradient = pd.read_csv('data/08-data/Lab08-MN-data.csv')

In [32]:
plt(data='gradient')

TypeError: 'module' object is not callable

## Deliverables
Submit an informal report detailing:

1. Your name, lab title
2. Description of the objectives and the field procedure
3. A section containing the processed Schlumberger sounding data and a qualitative interpretation.
    * How does the apparent resistivity change with AB/2?
    * What might this suggest about how the resistivity of Kafadar changes with depth? 
4. A section containing the processed gradient data and a qualitative interpretation.
    * How does the apparent resistivity change laterally?
    * Are there any points of interest in the survey?
5. A concluding section summarizing your work and speculating how to use these two types of survey combined to tackle a problem that requires understanding of 3D subsurface geo-electrical structure.
